In [2]:
import tkinter as tk
import mysql.connector
from mysql.connector import Error

# Create a connection to the MySQL database
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='11Mfld11',
            database='mystic_schema'
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None

In [10]:
# Create a new chat room with error handling


def create_chat_room():
    room_name = room_name_entry.get()
    creator_id = creator_id_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Check if the creator_id exists in the Characters table
        cursor.execute("SELECT id FROM Characters WHERE id = %s", (creator_id,))
        user_exists = cursor.fetchone()

        if user_exists:
            # Insert a new chat room
            insert_query = "INSERT INTO ChatRooms (ChatRoomName, CreatedByUserID) VALUES (%s, %s)"
            cursor.execute(insert_query, (room_name, creator_id))
            connection.commit()

            # Get the ChatRoomID of the newly created chat room
            cursor.execute("SELECT ChatRoomID FROM ChatRooms WHERE ChatRoomName = %s", (room_name,))
            chat_room_id = cursor.fetchone()[0]

            # Insert the creator into the ChatRoomMembers table
            insert_member_query = "INSERT INTO ChatRoomMembers (ChatRoomID, UserID) VALUES (%s, %s)"
            cursor.execute(insert_member_query, (chat_room_id, creator_id))
            connection.commit()

            cursor.close()
            connection.close()
        else:
            print(f"User with ID {creator_id} does not exist in the Characters table.")

# Create the Tkinter GUI window
root = tk.Tk()
root.title("Chat Room Manager")

# Create labels and entry fields for chat room creation
room_name_label = tk.Label(root, text="Chat Room Name:")
room_name_label.pack()
room_name_entry = tk.Entry(root)
room_name_entry.pack()

creator_id_label = tk.Label(root, text="Created By (User ID):")
creator_id_label.pack()
creator_id_entry = tk.Entry(root)
creator_id_entry.pack()

create_button = tk.Button(root, text="Create Chat Room", command=create_chat_room)
create_button.pack()

root.mainloop()

In [17]:
#Chat Room Joining and Writing with Error Handling

import tkinter as tk

# Function to fetch existing chat rooms
def fetch_chat_rooms():
    connection = create_connection()
    chat_rooms = []

    if connection:
        cursor = connection.cursor()

        # Fetch existing chat rooms from the ChatRooms table
        cursor.execute("SELECT ChatRoomName FROM ChatRooms")
        chat_rooms = [row[0] for row in cursor.fetchall()]

        cursor.close()
        connection.close()

    return chat_rooms



# Function to join an existing chat room
def join_chat_room():
    selected_room = chat_room_listbox.get(tk.ACTIVE)
    user_id = user_id_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Check if the user with the given User ID exists in the Characters table
        cursor.execute("SELECT id FROM Characters WHERE id = %s", (user_id,))
        user_exists = cursor.fetchone()

        if user_exists:
            # Get the ChatRoomID based on the selected chat room name
            cursor.execute("SELECT ChatRoomID FROM ChatRooms WHERE ChatRoomName = %s", (selected_room,))
            chat_room_id = cursor.fetchone()[0]

            # Check if the user is already in the ChatRoomMembers table for this specific chat room
            cursor.execute("SELECT COUNT(*) FROM ChatRoomMembers WHERE ChatRoomID = %s AND UserID = %s", (chat_room_id, user_id))
            count = cursor.fetchone()[0]

            if count == 0:
                # Insert the user into the ChatRoomMembers table
                insert_query = "INSERT INTO ChatRoomMembers (ChatRoomID, UserID) VALUES (%s, %s)"
                cursor.execute(insert_query, (chat_room_id, user_id))
                connection.commit()
            else:
                print(f"User with ID {user_id} is already a member of this chat room.")
        else:
            print(f"User with ID {user_id} does not exist.")

        cursor.close()
        connection.close()

# Function to send a message to the selected chat room
def send_message():
    selected_room = chat_room_listbox.get(tk.ACTIVE)
    user_id = user_id_entry.get()
    message_text = message_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Check if the user with the given User ID exists in the Characters table
        cursor.execute("SELECT id FROM Characters WHERE id = %s", (user_id,))
        user_exists = cursor.fetchone()

        if user_exists:
            # Get the ChatRoomID based on the selected chat room name
            cursor.execute("SELECT ChatRoomID FROM ChatRooms WHERE ChatRoomName = %s", (selected_room,))
            chat_room_id = cursor.fetchone()[0]

            # Check if the user is a member of the ChatRoomMembers table for this specific chat room
            cursor.execute("SELECT COUNT(*) FROM ChatRoomMembers WHERE ChatRoomID = %s AND UserID = %s", (chat_room_id, user_id))
            count = cursor.fetchone()[0]

            if count > 0:
                # Insert the message into the ChatMessages table
                insert_query = "INSERT INTO ChatMessages (ChatRoomID, UserID, MessageContent) VALUES (%s, %s, %s)"
                cursor.execute(insert_query, (chat_room_id, user_id, message_text))
                connection.commit()
            else:
                print(f"User with ID {user_id} is not a member of this chat room.")
        else:
            print(f"User with ID {user_id} does not exist.")

        cursor.close()
        connection.close()


# Create the Tkinter GUI window
root = tk.Tk()
root.title("Join Chat Room")

# Fetch existing chat rooms
chat_rooms = fetch_chat_rooms()

# Create a listbox to display available chat rooms
chat_room_listbox = tk.Listbox(root)
chat_room_listbox.pack()

# Populate the chat room listbox with existing chat rooms
for room in chat_rooms:
    chat_room_listbox.insert(tk.END, room)

# Create labels and entry fields for joining chat rooms
user_id_label = tk.Label(root, text="Your User ID:")
user_id_label.pack()
user_id_entry = tk.Entry(root)
user_id_entry.pack()

join_button = tk.Button(root, text="Join Chat Room", command=join_chat_room)
join_button.pack()

# Create labels and entry fields for sending messages
message_label = tk.Label(root, text="Message:")
message_label.pack()
message_entry = tk.Entry(root)
message_entry.pack()

send_button = tk.Button(root, text="Send Message", command=send_message)
send_button.pack()


root.mainloop()


User with ID 1 is already a member of this chat room.


In [4]:
#private chat room


import tkinter as tk
import mysql.connector
from mysql.connector import Error

# Create a connection to the MySQL database
def create_connection():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='root',
            password='11Mfld11',
            database='mystic_schema'
        )
        return connection
    except Error as e:
        print(f"Error: {e}")
        return None


import tkinter as tk

# Function to send a private message
def send_private_message():
    player1_id = player1_id_entry.get()
    player2_id = player2_id_entry.get()
    message_text = private_message_entry.get()

    connection = create_connection()
    if connection:
        cursor = connection.cursor()

        # Check if both player IDs exist in the Characters table
        cursor.execute("SELECT id FROM Characters WHERE id = %s", (player1_id,))
        player1_exists = cursor.fetchone()

        cursor.execute("SELECT id FROM Characters WHERE id = %s", (player2_id,))
        player2_exists = cursor.fetchone()

        if player1_exists and player2_exists:
            # Get the last DialogueID
            cursor.execute("SELECT MAX(DialogueID) FROM Dialogue")
            last_dialogue_id = cursor.fetchone()[0]
            if last_dialogue_id is None:
                last_dialogue_id = 0

            # Calculate the new DialogueID
            new_dialogue_id = last_dialogue_id + 1

            # Insert the private message into the Dialogue table with the new DialogueID and timestamp
            insert_query = "INSERT INTO Dialogue (DialogueID, FirstPlayerID, SecondPlayerID, Text, Timestamp) " \
                           "VALUES (%s, %s, %s, %s, CURRENT_TIMESTAMP)"
            cursor.execute(insert_query, (new_dialogue_id, player1_id, player2_id, message_text))
            connection.commit()
        else:
            if not player1_exists:
                print(f"Player with ID {player1_id} does not exist.")
            if not player2_exists:
                print(f"Player with ID {player2_id} does not exist.")

        cursor.close()
        connection.close()

# Create the Tkinter GUI window for private messaging
private_chat_window = tk.Tk()
private_chat_window.title("Private Chat")

# Create labels and entry fields for player IDs and private messages
player1_id_label = tk.Label(private_chat_window, text="Player 1 ID:")
player1_id_label.pack()
player1_id_entry = tk.Entry(private_chat_window)
player1_id_entry.pack()

player2_id_label = tk.Label(private_chat_window, text="Player 2 ID:")
player2_id_label.pack()
player2_id_entry = tk.Entry(private_chat_window)
player2_id_entry.pack()

private_message_label = tk.Label(private_chat_window, text="Message:")
private_message_label.pack()
private_message_entry = tk.Entry(private_chat_window)
private_message_entry.pack()

send_private_button = tk.Button(private_chat_window, text="Send Private Message", command=send_private_message)
send_private_button.pack()

private_chat_window.mainloop()


